# DeGiro analyser for stocks and ETFs
This notebook generates plots that display (1) monthly and (2) cumulative purchasing of assets, and (3) the profit/loss compared to the current asset price.

The following nomenclature is used for ETFs and stocks
- Quantity: the amount of a particular asset
- Value: the value of an asset in a defined currency
- Total: Quantity * Value

In [ ]:
# load libraries
import pandas as pd
from src.functions import get_quantity_and_value_from_description
from src.functions import read_account_overview
from src.functions import get_start_datetime
from src.functions import get_historical_stock_price
from src.plots import plot_dividend, plot_transactions
from src.plots import plot_cumulative_transaction_value_and_current_value
from datetime import datetime, timedelta
import matplotlib.pyplot as plt


In [ ]:
# Load account xls into dataframe, calculate reduced (YYYY-MM)
df_account = read_account_overview('Account-2.xls')

# Define product and ticker symbols as dictionary 
products = {
    'VANGUARD': 'VWRL.AS',
    'MATERIALS': 'MP'
}

In [ ]:
df_transactions

In [ ]:
for product, ticker in products.items():
    # Get only records of pre-defined product
    df_product = df_account[df_account['Product'].str.contains(product)]

    # Selection of all transaction of product
    df_transactions = df_product[df_product['Omschrijving'].str.contains('Koop')].copy()

    # Get the currency of the account
    currency = df_transactions['Mutatie_Valuta'].values[0]
    
    # Get the quantity and value of each transaction
    df_transactions[['Transaction_Quantity', 'Transaction_Value']] = df_transactions['Omschrijving'].apply(lambda x: get_quantity_and_value_from_description(x)).tolist()

    # Calculate the total value of each transaction (quantity * value)
    df_transactions['Transaction_Total'] = df_transactions['Transaction_Quantity'] * df_transactions['Transaction_Value']
    
    # Group all transactions by month and calculate the cumulative sum
    columns = ['Transaction_Quantity', 'Transaction_Value', 'Transaction_Total']
    df_transactions = df_transactions.groupby('Datum_Year_Month', as_index=False)[columns].sum()

    # Calculate the cumulatice sum of the transaction quantity, total and value
    # Important: the Transaction_Total_Cum * Transaction_Quantity_Cum does NOT equals the Transaction_Value_Cum
    df_transactions['Transaction_Quantity_Cum'] = df_transactions['Transaction_Quantity'].cumsum(axis=0)
    df_transactions['Transaction_Total_Cum'] = df_transactions['Transaction_Total'].cumsum(axis=0)
    df_transactions['Transaction_Value_Cum'] = df_transactions['Transaction_Value'].cumsum(axis=0)
    
    plot_transactions(product,
                    df_transactions['Datum_Year_Month'],
                    df_transactions['Transaction_Total'],
                    df_transactions['Transaction_Total_Cum'],
                    currency)

    # This is not the actual received dividend
    # One must subtract the DEGIRO Corporate Action Kosten and consider foreign exchange conversion.
    # To Be Done
    dividend = df_product[df_product['Omschrijving'].str.contains('Dividend')].copy()

    # Check if there are any dividends 
    # - Some stocks do not bring out dividends (yet)
    # - Some etf's are accumulating and reinvest dividends directly
    if dividend.empty:
        print(f'No dividend found for {product}')
    else:
        dividend['Mutatie_Bedrag_Cum'] = dividend['Mutatie_Bedrag'].cumsum(axis=0)
        currency = dividend['Mutatie_Valuta'].values[0]
        
        plot_dividend(product,
                    dividend['Datum_Year_Month'],
                    dividend['Mutatie_Bedrag'],
                    dividend['Mutatie_Bedrag_Cum'],
                    currency)
        
        dividend.groupby('Datum_Year')['Mutatie_Bedrag'].describe()

    # Define start and end date to get historical stock price
    start_datetime = get_start_datetime(df_product['Datum'].min())
    current_datetime = datetime.now().strftime('%Y-%m-%d')

    # Get historical stock price and reduce to monthly data
    df_historical_price = get_historical_stock_price(ticker, start_datetime, current_datetime, '1mo')

    # Filter the stock price data to only include the dates of the transactions
    filtered_df_ticker = df_historical_price[df_historical_price['Datum_Year_Month'].isin(df_transactions['Datum_Year_Month'])]
    
    # Remove first row because we want to use the last day of the month (= first day of next month) as the stock price
    filtered_df_ticker = filtered_df_ticker[1:]
    
    # Subtract 2 days to make sure the stock price is available (in case the code is run on a weekend)
    start_datetime_minus_2days = datetime.now() - timedelta(days=2)
    
    # Get the last record of the dataframe as this corresponds to the most recent stock price
    current_price = get_historical_stock_price(ticker, start_datetime_minus_2days, current_datetime, '1d').tail(1)
    
    # Add the most recent stock price to the filtered dataframe
    filtered_df_ticker = pd.concat([filtered_df_ticker, current_price])

    # Add historical prices to transactions for comparison
    df_transactions['Historical_Value'] = filtered_df_ticker['Close'].values
    df_transactions['Stock_Value_Cum'] = df_transactions['Transaction_Quantity_Cum'] * df_transactions['Historical_Value']
    
    plot_cumulative_transaction_value_and_current_value(
        ticker,
        df_transactions['Datum_Year_Month'],
        df_transactions['Transaction_Total_Cum'],
        df_transactions['Stock_Value_Cum'],
        currency)
    